In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image,ImageOps
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import os
os.chdir('/home/omkar/CNN proj')




2021-11-27 22:49:55.631484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omkar/.local/lib/python3.8/site-packages/cv2/../../lib64:
2021-11-27 22:49:55.631517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
training_data=[]
classes=43
cur_path = os.getcwd()
for i in range(classes):     
    path = os.path.join(cur_path,'train',str(i))     
    images = os.listdir(path)
    for a in images:
        try:
            
            img_array = cv2.imread(path +'/'+ a,cv2.IMREAD_GRAYSCALE)  # convert to array
            new_array = cv2.resize(img_array, (30, 30))
            training_data.append([new_array,i])
        
        except Exception as e:
            print(e)

In [3]:
print(len(training_data))

39209


In [4]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)


X = np.array(X).reshape(-1, 30, 30, 1)
y = np.array(y)

In [5]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 

(31367, 30, 30, 1) (7842, 30, 30, 1) (31367,) (7842,)


In [6]:
y_train = to_categorical(y_train,43) 
y_test = to_categorical(y_test,43)

In [7]:
y_train.shape,y_test.shape

((31367, 43), (7842, 43))

In [8]:
X = X/255.0

In [9]:
model =Sequential() 
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:])) 
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2))) 
model.add(Dropout(rate=0.25)) 

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu')) 
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25)) 

model.add(Flatten()) 

model.add(Dense(256, activation='relu')) 
model.add(Dropout(rate=0.5))

# We have 43 classes that's why we have defined 43in the dense 
model.add(Dense(43, activation='softmax'))

2021-11-27 22:51:11.466847: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-27 22:51:11.466897: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (omkar-Lenovo-ideapad-520-15IKB): /proc/driver/nvidia/version does not exist
2021-11-27 22:51:11.467639: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#Compilation of the model 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))

2021-11-27 22:51:11.839173: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 28230300 exceeds 10% of free system memory.
2021-11-27 22:51:12.638859: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2021-11-27 22:51:15.148395: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 29427200 exceeds 10% of free system memory.
2021-11-27 22:51:15.149030: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 29427200 exceeds 10% of free system memory.


  2/981 [..............................] - ETA: 55s - loss: 21.8832 - accuracy: 0.0156  

2021-11-27 22:51:15.577746: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 29427200 exceeds 10% of free system memory.
2021-11-27 22:51:15.580001: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 29427200 exceeds 10% of free system memory.


981/981 [==============================] - 78s 76ms/step - loss: 1.4736 - accuracy: 0.6232 - val_loss: 0.2423 - val_accuracy: 0.9426
Epoch 2/20
981/981 [==============================] - 63s 64ms/step - loss: 0.3500 - accuracy: 0.9018 - val_loss: 0.1890 - val_accuracy: 0.9543
Epoch 3/20
981/981 [==============================] - 69s 70ms/step - loss: 0.2436 - accuracy: 0.9320 - val_loss: 0.0752 - val_accuracy: 0.9797
Epoch 4/20
981/981 [==============================] - 71s 72ms/step - loss: 0.1901 - accuracy: 0.9467 - val_loss: 0.0533 - val_accuracy: 0.9858
Epoch 5/20
981/981 [==============================] - 70s 71ms/step - loss: 0.1891 - accuracy: 0.9481 - val_loss: 0.0680 - val_accuracy: 0.9848
Epoch 6/20
981/981 [==============================] - 69s 70ms/step - loss: 0.1696 - accuracy: 0.9523 - val_loss: 0.0348 - val_accuracy: 0.9911
Epoch 7/20
981/981 [==============================] - 61s 62ms/step - loss: 0.1517 - accuracy: 0.9583 - val_loss: 0.0452 - val_accuracy: 0.9876
Epo

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
model.save("./TSR.h5")

In [ ]:
df = pd.DataFrame(history.history)
df

In [ ]:
print(f"Accuracy: {df['val_accuracy'][19]*100} %")